In [12]:
using JuMP, Gurobi, LinearAlgebra, Statistics, Plots

In [13]:
#time step
T=[1,2,3];
#scenarios
S=Dict();S[1]=1;S[2]=1:2;S[3]=1:4;
#probability
p=Dict();p[1]=1;p[2]=[1/2,1/2];p[3]=[1/4,1/4,1/4,1/4];
#parent node
a=Dict()
a[3,1]=[2,1]
a[3,2]=[2,1]
a[3,3]=[2,2]
a[3,4]=[2,2]
a[2,1]=[1,1]
a[2,2]=[1,1]
a[1,1]=nothing
#demand
d=Dict();d[1]=800;d[2]=[1200, 400];d[3]=[1600,1000,600,200];
#decision stage
D=[1,2];
#capacity and cost list
B=[1000,1500];C=[1145,1500];N=length(C);
#limitations, costs and prices
x_bar=1500;y_bar=2000;s_bar=400
π_p=140;ρ_p=50;ρ_s=30;ρ_w=30;
#constant mean profit level
μ_c=70000;
γ=0.0;

In [11]:
m=Model(optimizer_with_attributes(Gurobi.Optimizer))

@variable(m, u[t in D, j in S[t], k in 1:N] >= 0, Int)
#selection of plants with different capacity
@expression(m, x[t in D,j in S[t]], sum(u[t,j,k] * B[k] for k in 1:N))
@expression(m, y[t in D,j in S[t]], sum(u[t,j,k] * C[k] for k in 1:N))

#storage and waste
@variable(m, 0 <= s[t in T, j in S[t]], Int)
@constraint(m, [t in D, j in S[t]], s[t,j] <= s_bar)
@variable(m, 0 <= w[t in T, j in S[t]], Int)

#cumulative variables
@variable(m, X[t in T, j in S[t]] <= x_bar)

# helper constraints
@constraint(m, X[1,S[1]] == 0)
@constraint(m, [t in T[2:end], j in S[t]], X[t,j] == X[a[t,j][1], a[t,j][2]] + x[a[t,j][1], a[t,j][2]])

#storing
@constraint(m, s[1,S[1]] == 0)
@constraint(m, [j in S[T[end]]], s[T[end],j] == 0)
@constraint(m, w[1,S[1]] == 0)
@constraint(m, [t in T[2:end], j in S[t]], X[t,j] + s[a[t,j][1], a[t,j][2]] - s[t,j] - w[t,j] <= d[t][j])  
@constraint(m, [t in T[2:end], j in S[t]], 0<=X[t,j] + s[a[t,j][1], a[t,j][2]] - s[t,j] - w[t,j]) 

#cost
@variable(m, q[t in T, j in S[t]])
@constraint(m, [t in D, j in S[t]], q[t,j]==y[t,j]+ρ_s*s[t,j]+ρ_p * X[t,j] + ρ_w*w[t,j])
@constraint(m, [j in S[T[end]]], q[T[end],j]==ρ_w*w[T[end],j]+ρ_p * X[T[end],j])
 
#revenue
@variable(m, r[t in T, j in S[t]])
@constraint(m, [j in S[1]], r[1,j]==0)
@constraint(m, [t in T[2:end], j in S[t]], r[t,j]==π_p * (X[t,j] + s[a[t,j][1], a[t,j][2]] - s[t,j] - w[t,j]))

#discounting
@expression(m, β[t in T], 1 / (1 + γ) ^ (t - 1))

#mean absolute deviation variables
@expression(m, v[t in T, j in S[t]], β[t]*(r[t,j]-q[t,j]))
@variable(m, V[t in T, j in S[t]])
@constraint(m, V[1,S[1]]==v[1,S[1]])
@constraint(m, [t in T[2:end], j in S[t]], V[t,j]==V[a[t,j][1], a[t,j][2]]+v[t,j])
@variable(m, R[T[end], j in S[T[end]]] >= 0)
@expression(m, ϵ, sum(p[T[end]][j]*V[T[end],j] for j in S[T[end]]))

#mean_profit
@constraint(m, ϵ >= μ_c)

#Mean absolute deviation
@constraint(m, [j in S[T[end]]], R[T[end],j] >= p[T[end]][j]*(V[T[end],j] - ϵ))
@constraint(m, [j in S[T[end]]], R[T[end],j] >= p[T[end]][j]*(ϵ - V[T[end],j]))

@objective(m, Min, sum(R[T[end],j] for j in S[T[end]]))
status=optimize!(m)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 58 rows, 52 columns and 214 nonzeros
Model fingerprint: 0x3bbc0b3b
Variable types: 32 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-02, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+03, 2e+03]
  RHS range        [2e+02, 7e+04]
Presolve removed 49 rows and 37 columns
Presolve time: 0.00s
Presolved: 9 rows, 15 columns, 35 nonzeros
Variable types: 0 continuous, 15 integer (0 binary)
Found heuristic solution: objective 108842.50000

Root relaxation: objective 9.514500e+04, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 95145.0000    0    1 108842.500 95145.0000  12.6%     -    0s
H    0     0                    95157.500000 951

In [15]:
println("The Profit is ", value.(ϵ))
println("The abosulte deviation is ", sum(value.(R[T[end],j]) for j in S[T[end]]))
for t in D
    for j in S[t]
        println("At stage ", t, " scenarios ", j, " we should install ", string(value.(x[t,j])),  " capacity")
        println("At stage ", t, " scenarios ", j, " we stored ", string(value.(s[t,j])),  " product")
        println("At stage ", t, " scenarios ", j, " we dumped ", string(value.(w[t,j])),  " product")
    end
end

for j in S[T[end]]
    println("The cumulative profit at final stage and scenarios ", j , " are ", value.(V[T[end],j]))
end

The Profit is 70000.0
The abosulte deviation is 95145.0
At stage 1 scenarios 1 we should install 1000.0 capacity
At stage 1 scenarios 1 we stored 0.0 product
At stage 1 scenarios 1 we dumped 0.0 product
At stage 2 scenarios 1 we should install 0.0 capacity
At stage 2 scenarios 1 we stored 15.0 product
At stage 2 scenarios 1 we dumped 70.0 product
At stage 2 scenarios 2 we should install 0.0 capacity
At stage 2 scenarios 2 we stored -0.0 product
At stage 2 scenarios 2 we dumped 600.0 product
The cumulative profit at final stage and scenarios 1 are 166335.0
The cumulative profit at final stage and scenarios 2 are 163955.0
The cumulative profit at final stage and scenarios 3 are 8855.0
The cumulative profit at final stage and scenarios 4 are -59145.0
